In [ ]:
import pandapower
import pandapower.networks
import pandapower.plotting
import pandapower.topology
import pandapower.converter
import pandapower.estimation

## Numpy has to be version 1.26.4 or lower, because they changed the name of the INF constant, also pyproj is required (not stated in the documentation)

In [ ]:
import pandapower.test
pandapower.test.run_all_tests()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandapower as pp
import os
import pandapower.topology as top
import pandapower.plotting as plot
import simbench as sb

In [ ]:
import pandapower.test
pandapower.test.run_all_tests()

In [ ]:
import pandapower.test
pandapower.test.run_all_tests()

In [ ]:
sb_code = "1-LV-semiurb4--0-sw"
net = sb.get_simbench_net(sb_code)

In [ ]:
net

In [ ]:
plot.simple_plot(net)

In [ ]:
net.loadcases

In [ ]:
loadcases = sb.get_absolute_values(net, profiles_instead_of_study_cases=True)

def apply_absolute_values(net, absolute_values_dict, case_or_time_step):
    for elm_param in absolute_values_dict.keys():
        if absolute_values_dict[elm_param].shape[1]:
            elm = elm_param[0]
            param = elm_param[1]
            net[elm].loc[:, param] = absolute_values_dict[elm_param].loc[case_or_time_step]

In [ ]:
net.sgen.profile.iloc[0]
assert not sb.profiles_are_missing(net)

In [ ]:
profiles = sb.get_absolute_values(net, profiles_instead_of_study_cases=True)
time_steps = range(96)
net.trafo.tap_pos = 1
results = pd.DataFrame([], index=time_steps, columns=["Load Sum", "min_vm_pu", "max_vm_pu"])
for time_step in time_steps:
    apply_absolute_values(net, profiles, time_step)
    pp.runpp(net)
    results.loc[time_step, "Load Sum"] = net.res_load.p_mw.sum()
    results.loc[time_step, "min_vm_pu"] = net.res_bus.vm_pu.min()
    results.loc[time_step, "max_vm_pu"] = net.res_bus.vm_pu.max()

In [ ]:
fig1, ax1 = plt.subplots()
results[["min_vm_pu", "max_vm_pu"]].plot(ax=ax1)
ax1.set_ylabel("Voltage Extrema [p.u.]")
ax1.set_xlabel("Time Step (quarter hour)")
plt.show()

## Using the Timeseries Module of pandapower

In [ ]:
import tempfile
from pandapower.timeseries import OutputWriter
from pandapower.timeseries.run_time_series import run_timeseries

sb.apply_const_controllers(net, profiles)
output_dir = os.path.join(tempfile.gettempdir(), "time_series_example")
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
ow = OutputWriter(net, time_steps, output_path=output_dir, output_file_type=".json")
ow.log_variable("res_load", "p_mw", eval_function=np.sum, eval_name="Load Sum")
ow.log_variable("res_bus", "vm_pu", eval_function=np.min, eval_name="min_vm_pu")
ow.log_variable("res_bus", "vm_pu", eval_function=np.max, eval_name="max_vm_pu")

run_timeseries(net, time_steps)

vm_pu_file = os.path.join(output_dir, "res_bus", "vm_pu.json")
vm_pu = pd.read_json(vm_pu_file)
load_p_file = os.path.join(output_dir, "res_load", "p_mw.json")
load_p = pd.read_json(load_p_file)
result = pd.concat([load_p[["Load Sum"]], vm_pu[["min_vm_pu", "max_vm_pu"]]], axis=1)
result.sort_index(inplace=True)

In [ ]:
results.head()

In [ ]:
fig1, ax1 = plt.subplots()
results[["min_vm_pu", "max_vm_pu"]].plot(ax=ax1)
ax1.set_ylabel("Voltage Extrema [p.u.]")
ax1.set_xlabel("Time Step (quarter hour)")
plt.show()

## Using MOSAIK

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import mosaik
import mosaik.util
import mosaik_pandapower

SIM_CONFIG = {
    'Grid': {
        'python': 'mosaik_pandapower.simulator:Pandapower',
    }
}
END = 600

world = mosaik.World(SIM_CONFIG)


GRID_FILE = "1-LV-semiurb4--0-sw"
gridsim = world.start('Grid', step_size=900)
grid = gridsim.Grid(gridfile=GRID_FILE).children

world.run(until=END)